# Conversational Finance

This dataset is composed of complicated financial questions, answerable with reference to tables and textual context. Either follow through the executed code in this notebook, or execute the code yourself to have a play around with the outputs of the models. Run the below cell once only to download the dataset, then comment it out again to prevent errors.

In [36]:
# !git clone https://github.com/czyssrs/ConvFinQA.git
# !unzip ConvFinQA/data.zip -d ConvFinQA/

## Section 1: Dataset analysis

Let's first preprocess the data and look at it in detail.

In [37]:
import json
from tqdm import tqdm

from utils.preprocessing import Preprocessor

with open("ConvFinQA/data/train.json", "r") as f:
    raw_data = json.load(f)

preprocessor = Preprocessor(raw_data)
data = preprocessor.preprocess()

In [ ]:
# A random question

import random

from utils.general import print_example

print_example(data[random.randint(0, len(data) - 1)])


  Example datapoint:
  Question: what was the percentage change in the non interest income from from 2011 to 2012
  Answer: 0.05357
  Predicted answer: Not yet predicted
  Table: +---------------------------------------------+-----------------------+-----------------------+
|   year ended december 31dollars in millions | net interest income   | net interest margin   |
+=============================================+=======================+=======================+
|                                        2012 | $ 9640                | 3.94% ( 3.94 % )      |
+---------------------------------------------+-----------------------+-----------------------+
|                                        2011 | $ 8700                | 3.92% ( 3.92 % )      |
+---------------------------------------------+-----------------------+-----------------------+
  


In [ ]:
# Aggregate Dataset statistics

from collections import Counter

print(f"Dataset length: {len(data)}")
counter = Counter([type(d.answer) for d in data])
print(f"Answer type counts: {counter}")
print(f"Possible string answers: {set([d.answer for d in data if isinstance(d.answer, str)])}")
print(f"Example float answer: {data[0].answer}")

Dataset length: 3965
Answer type counts: Counter({<class 'float'>: 3921, <class 'str'>: 44})
Possible string answers: {'yes', 'no'}
Example float answer: 0.14136


The vast majority of the examples in the data have floats as answers. I will constrain this investigation into question answering for these as opposed to the minority of 'yes/no' questions.

In [39]:
data = [d for d in data if isinstance(d.answer, float)]

print(f"Dataset length after filtering: {len(data)}")

Dataset length after filtering: 3921


With this filtering done, I want to go through three different kinds of example I saw:

1. Good context, good answer (Ideal)
2. Bad context, good answer (Underinformed)
3. Good context, bad answer (Incorrectly labelled)

### Ideal

In the below example, we can see a question, answer, and the table used to answer the question. (Note that there is also context I have chosen not to print out for the sake of focus.) This kind of example is the *ideal* type within our dataset: the required data is present in the table and the gold standard answer is correct. We can generate an answer to this question and compare it to a gold standard to evaluate the quality of our system effectively. A comparison between a predicted answer and gold standard answer will be *meaningful*

In [ ]:
print_example(data[1])


  Example datapoint:
  Question: what was the percent of the growth in the revenues from 2007 to 2008
  Answer: 0.01269
  Predicted answer: Not yet predicted
  Table: +-------------------------------------------+-----------+----------------------------------------------------------------------+----------------------------+------------------------------+
|                                           | revenue   |   income from continuing operations available to common stockholders |   basic earnings per share |   diluted earnings per share |
+===========================================+===========+======================================================================+============================+==============================+
| year ended december 31 2008 ( unaudited ) | $ 9362.2  |                                                                285.7 |                       0.76 |                         0.75 |
+-------------------------------------------+-----------+-------------------

### Underinformed

On the other hand, consider the example shown below. The data is corrupted in some way, such that two rows are marked as being the 'year ended june 30 2009 2008'. It is not really clear how we should answer this question. I take the view that, in fact, in a production question-answering system, we **should not** answer this question, but request clarification or data cleaning from the end-user. For this initial analysis application, I have chosen to output a flag, **CANNOT ANSWER**, which could be picked up in an end-to-end chatbot.

In [41]:
print_example(data[0])


  Example datapoint:
  Question: what was the percentage change in the net cash from operating activities from 2008 to 2009
  Answer: 0.14136
  Predicted answer: Not yet predicted
  Table: +------------------------------+--------------+---------------------+-------------------------+------------------------------+------------------------------------------+--------------------------------------+
| 2008                         | net income   |   non-cash expenses | change in receivables   |   change in deferred revenue | change in other assets and liabilities   | net cash from operating activities   |
+==============================+==============+=====================+=========================+==============================+==========================================+======================================+
| year ended june 30 2009 2008 | $ 103102     |               74397 | 21214                   |                        21943 | -14068 ( 14068 )                         | $ 206588     

### Incorrectly labelled

In the below example, data required to answer the question is present but the gold standard label is incorrect. In this case, the portion of the total shares subject to outstanding awards is *clearly* 2530454 / (2530454 + 5923147) = ~0.29, not the gold standard answer which puts the 2004 shares in the numerator. These questions are dangerous in our dataset. In fact, they represent an automatic performance penalty to any hypothetically perfect model. Any satisfactory analysis should take care that mismatches between predicted answers and incorrectly labelled answers are, at the very least, not marking down system performance.

In [42]:
print_example(data[6])


  Example datapoint:
  Question: what portion of the total shares subject to outstanding awards is under the 2009 global incentive plan?
  Answer: 0.70067
  Predicted answer: Not yet predicted
  Table: +--------------------------------------+------------------------------+-----------------------------+
|                                      |   2009 global incentive plan | 2004 stock incentive plan   |
+======================================+==============================+=============================+
| shares available for awards          |                      2322450 | -                           |
+--------------------------------------+------------------------------+-----------------------------+
| shares subject to outstanding awards |                      2530454 | 5923147                     |
+--------------------------------------+------------------------------+-----------------------------+
  


### Section 1 Summary

We have different sources of data which we need to handle. We should primarily measure system accuracy using *ideal* questions, not *underinformed* or *incorrectly labelled*.

## The implementation

This dataset is highly amenable to usage with LLMs. Each question requires combining some context and domain knowledge to answer. Because the context is fairly small, we don't need to do any context retrieval to derive the answer to each question. The information needed to answer them is all included in relatively short windows of text and structured tables. If we structure this information the right way, and provide it to a sufficiently powerful LLM with enough domain knowledge, it should be able to answer the questions (at least the ideal ones).

## Baseline build

The baseline was simply a model instructed to directly answer the question, provided the table and context. This model is linked below, and resulted in initial accuracies of around 40%.

In [ ]:
from openai import OpenAI

OPENAI_API_KEY=""

client = OpenAI(api_key=OPENAI_API_KEY)

In [44]:
from agents.question_answering import QuestionAnswerer
from agents.prompts.question_answering import first_pass

first_pass_agent = QuestionAnswerer(client=client, model='gpt-4o', system_prompt=first_pass.system_prompt, user_prompt=first_pass.user_prompt)
answered_example = first_pass_agent.predict(data[2])

print_example(answered_example)


  Example datapoint:
  Question: what was the percentage change in net sales from 2000 to 2001?
  Answer: -0.3282
  Predicted answer: -32.81
  Table: +------+-------------+-----------------+----------------+---------------------------+
|      | net sales   |   cost of sales | gross margin   | gross margin percentage   |
+======+=============+=================+================+===========================+
| 2002 | $ 5742      |            4139 | $ 1603         | 28% ( 28 % )              |
+------+-------------+-----------------+----------------+---------------------------+
| 2001 | $ 5363      |            4128 | $ 1235         | 23% ( 23 % )              |
+------+-------------+-----------------+----------------+---------------------------+
| 2000 | $ 7983      |            5817 | $ 2166         | 27% ( 27 % )              |
+------+-------------+-----------------+----------------+---------------------------+
  


## Improvements

To improve the model, I sectioned off my data into a training set, which could be experimented with and used to optimise the model, and a validation set, which would only be run at the very end of optimisation to ensure I was not overfitting the model to the dataset. The main changes are listed below.

### Model output change

A large issue with the baseline system is that while LLMs are excellent retrievers and organisers of information, LLMs frequently make mathematical errors, particularly when generating precise floating point representations of numbers. This creates a source of mismatch between answers.

Given these results, and inspired by the original method pursued in the paper, I asked the model to simply retrieve the necessary numbers and output a mathematical expression which could be extracted and computed precisely after retrieval. This yielded much better results.

### Standardised units

The outputs in the gold standard can be a bit inconsistent. Most of the time, however, they follow a pattern. For example, percentages are usually referenced in decimals (although occasionally are represented as integer values). I attempt to follow the rough standard within the dataset, but will also do some fuzzy evaluation later on to account for the inconsistency in the training data.

### Applying correct formulas

I accounted for a standard error type, such as when calculating rates of growth, that I saw during optimisation. If I was to continue this optimisation, I would find other common errors and also add some prompting to guide the model towards the correct behaviour over time.

To see these optimisations in the prompt, uncomment and run the below cell

In [ ]:
# from agents.prompts.question_answering.final import system_prompt

# print(system_prompt)

In [45]:
training_set, validation_set = data[:-1000], data[-1000:]

In [46]:
question_answerer = QuestionAnswerer(client=client, model='gpt-4o')
answered_example = question_answerer.predict(data[2])

print_example(answered_example)


  Example datapoint:
  Question: what was the percentage change in net sales from 2000 to 2001?
  Answer: -0.3282
  Predicted answer: -0.3281974195164725
  Table: +------+-------------+-----------------+----------------+---------------------------+
|      | net sales   |   cost of sales | gross margin   | gross margin percentage   |
+======+=============+=================+================+===========================+
| 2002 | $ 5742      |            4139 | $ 1603         | 28% ( 28 % )              |
+------+-------------+-----------------+----------------+---------------------------+
| 2001 | $ 5363      |            4128 | $ 1235         | 23% ( 23 % )              |
+------+-------------+-----------------+----------------+---------------------------+
| 2000 | $ 7983      |            5817 | $ 2166         | 27% ( 27 % )              |
+------+-------------+-----------------+----------------+---------------------------+
  


In [47]:
print(f"Gold standard answer: {answered_example.answer}\nPredicted answer: {answered_example.predicted_answer}")

Gold standard answer: -0.3282
Predicted answer: -0.3281974195164725


As you can see these answers are now indeed the same, with the exception of a rounding issue. I define an evaluator class to take account of this, by rounding to the gold standard answer's level.

In [48]:
from evaluator import Evaluator

evaluator = Evaluator()
evaluator.accuracy([data[2]])

1.0

## Evaluation

While we are already handling rounding errors, there are a few other common mismatches between the gold standard output and the predicted output. It is often the case, for example, that the outputs are represented in different unit sizes, even accounting for overall trends. 

For example, if in the context amounts of money are referred to in the millions (such that 3 then refers to 3 million) the gold standard label could possibly be represented in the condensed format or its full form (3 or 3000000). Because of this, sometimes the model's output will misalign with the gold standard label. 

In [49]:
# I use some data I already passed through the model here, but run it with the final model if you'd like

import pickle

with open("pregenerated_examples/training_examples_3.pkl", "rb") as f:
    pre_annotated_data = pickle.load(f)

pre_annotated_data = [
    d for d in pre_annotated_data if isinstance(d.answer, float)
]

pre_annotated_data = [d for d in pre_annotated_data if d.predicted_answer is not None]

print(f"Dataset length after filtering: {len(pre_annotated_data)}")
print(f"Dataset accuracy: {evaluator.accuracy(pre_annotated_data)}")

incorrect_examples = evaluator.return_incorrect(pre_annotated_data)
print(f"Incorrect examples: {len(incorrect_examples)}")

Dataset length after filtering: 1041
Dataset accuracy: 0.7435158501440923
Incorrect examples: 243


In [50]:
errors = [incorrect_examples[i] for i in [6, 15, 44]]

for example in errors:
    print_example(example)


  Example datapoint:
  Question: what is the percentual decrease observed in the future minimum rental payments during 2008 and 2009?
  Answer: -0.13249
  Predicted answer: 0.13249211356466878
  Table: +--------+--------+--------+--------+--------+---------------+-----------------------------------+
| 2008   |   2009 |   2010 |   2011 |   2012 |   later years | total minimum payments required   |
+========+========+========+========+========+===============+===================================+
| $ 317  |    275 |    236 |    214 |    191 |           597 | $ 1830                            |
+--------+--------+--------+--------+--------+---------------+-----------------------------------+
  

  Example datapoint:
  Question: what percent did the realized and unrealized losses effect the assets as of 2008?
  Answer: 0.3347
  Predicted answer: -0.4846066134549601
  Table: +--------------+--------------------+--------------------------------------------------+-----------------------------

What do these errors show? They show a difference in output format, where the calculated answer is not the same. 

The first shows a difference in sign. Because the question already implied a downward shift, the predicted answer leaves out the negative sign. This seems not like an error.

The second shows a difference in unit. The answer is requested in millions, but the question was answered in billions. This is a legitimate error.

The third shows a difference in representing percentages. Here, the gold standard answer bucks the trend throughout the rest of the dataset of representing percentages as decimals. This does not feel like an error.

Because of the inconsistency in the dataset and the predicted answers I propose an alternative metric whereby we 'fuzzily' match answers and ignore these differences in scale and direction. This will result in a metric that captures if we've done the maths right. This metric will also, however, leave open the possibility that the answer is formatted in the wrong way - as can be seen in the second example.

This results in a score which tends to go up about 5%.

In [51]:
print(f"Non-fuzzy (strict) matching: {evaluator.accuracy(pre_annotated_data, fuzzy=False)}")
print(f"Fuzzy (permissive) matching: {evaluator.accuracy(pre_annotated_data, fuzzy=True)}")

Non-fuzzy (strict) matching: 0.7435158501440923
Fuzzy (permissive) matching: 0.8001921229586936


This leaves the 20% of examples where the predicted and gold standard answer *do not match at all*. The immediate conclusion to jump to is that 20% of our predictions are wrong. However, recall our inaccurately labeled examples. If our model correctly predicts the answer for these examples the model will be marked as incorrect. Indeed, see the example cited during our data analysis, where indeed our model predicts the correct result creating a mismatch with the incorrect label. 

In [52]:
question_answerer.predict(data[6])
print_example(data[6])


  Example datapoint:
  Question: what portion of the total shares subject to outstanding awards is under the 2009 global incentive plan?
  Answer: 0.70067
  Predicted answer: 0.2993344493074608
  Table: +--------------------------------------+------------------------------+-----------------------------+
|                                      |   2009 global incentive plan | 2004 stock incentive plan   |
+======================================+==============================+=============================+
| shares available for awards          |                      2322450 | -                           |
+--------------------------------------+------------------------------+-----------------------------+
| shares subject to outstanding awards |                      2530454 | 5923147                     |
+--------------------------------------+------------------------------+-----------------------------+
  


### Answer Checking

I've defined an LLM agent which has the same available information and then compares the gold standard answer with the predicted answer, and chooses a preferred answer (while explaining it's reasoning). I use a different LLM to do this (DeepSeek's V3 model) so that OpenAI isn't 'marking its own homework'. This relies on LLMs' ability to alter their reasoning when they have the correct answer provided to them. We can then use this to get a sense of how often our predicted answers are actually wrong.

Of course, this is a risky method of evaluation. It is completely possible that *neither* the gold standard nor the predicted answer are correct, or that the answer checker makes the wrong evaluation. This metric might therefore be used best to establish a potential 'performance ceiling' or 'band' in which the true performance might exist.

In [ ]:
from agents.answer_checker import AnswerChecker

DEEPSEEK_API_KEY = ""

deep_seek_client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")

answer_checker = AnswerChecker(deep_seek_client, model='deepseek-chat')

In [ ]:
## Uncomment this to run the answer checker on the incorrect examples

# from collections import Counter

# for eg in tqdm(incorrect_examples):
#     answer_checker.predict(eg)

# count = Counter([d.preferred_answer for d in incorrec t_examples])
# print(count)

## Validation Evaluation

I have been working with approximately the first 2000 rows of train.json. For the purposes of this experiment, I will use the last 1000 rows as a held out, final evaluation set.

In [55]:
# for i in tqdm(range(len(validation_set))):
#     question_answerer.predict(validation_set[i])

with open("pregenerated_examples/validation_set.pkl", "rb") as f: # just load them in if you don't want to wait a few hours
    validation_set = pickle.load(f)

validation_set = [d for d in validation_set if isinstance(d.answer, float)]

for i in range(len(validation_set)):
    if validation_set[i].predicted_answer is None:
        validation_set[i].predicted_answer = "CANNOT ANSWER"
    

print(f"Validation set length: {len(validation_set)}")


Validation set length: 996


In [56]:
print(evaluator.accuracy(validation_set, fuzzy=False))
print(evaluator.accuracy(validation_set, fuzzy=True))

0.7339357429718876
0.8012048192771084


In [57]:
incorrect_validation_examples = evaluator.return_incorrect(validation_set)

# for example in tqdm(incorrect_validation_examples):
#     answer_checker.predict(example)

with open("pregenerated_examples/incorrect_validation_examples.pkl", "rb") as f: # just load them in if you don't want to wait a few hours
    incorrect_validation_examples = pickle.load(f)

In [58]:
from collections import Counter

count = Counter([d.preferred_answer for d in incorrect_validation_examples])
print(count)

Counter({'predicted': 159, 'gold': 95})


By our most fuzzy metric, that is, if we assume that all of the times our evaluator preferred the predicted answer over the generated answer alongside our other fuzzing (accounting for mismatching signs, and different scales), the model is over 90% accurate.

It should be said that this is unlikely to be the true measure of performance. A more conservative performance estimate might be made by taking out the examples marked by the answer checker as favouring the predicted answer.

In [59]:
print(f"Most generous accuracy: {(1 - (count['gold']/len(validation_set))) * 100:.2f}%")
print(f"Accuracy removing ambiguous cases: {(1 - (count['gold']/(len(validation_set) - count['predicted']))) * 100:.2f}%")

Most generous accuracy: 90.46%
Accuracy removing ambiguous cases: 88.65%


## Conclusion

These results show a wide band of possible performance from our model. Around 75% of answers are exactly answered with this application. By being more flexible in this, accounting for variation in how the questions are answered, 80% are answered. Attempting to remove for incorrectly labelled examples, we saw total possible accuracy as being 90.39%, or by removing ambiguous examples we see fuzzy accuracy as 88.52%. 

For future work, the 10% of gold standard answers that the Answer Checker preferred potentially represent the most challenging, interesting examples that future projects need to optimise for. Additionally I decided for the purposes of this project that simply *not answering* (in the case where the model decided the context was too uninformative) was not an incorrect response or a correct response. Future possible investigations should evaluate the decisions to not answer, and see whether they were legitimate. After all, they could just represent very hard, interesting questions that require more domain knowledge.